<a href="https://colab.research.google.com/github/XescLlabres/tfgFLM/blob/main/HUSE_per_pacient/GowerSMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
data1 = pd.read_csv('df_gower.csv')

In [4]:
data1.drop(['ID', 'CLAVE'], axis=1)

,RECIST,GENDER,AGE_AT_TACE,DAYS_CT_PRETACE,WEIGHT,HEIGHT,BMI,BMI_CATEGORY,HCV,Alcohol _abuse\n,...,Tumoral_Distribution,I,II,III,IVa,IVb,V,VI,VII,VIII
0,0,1,82,47,64.0,1.47,29.62,3.0,1.0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,71,26,55.0,1.52,23.81,2.0,1.0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,48,75,65.0,1.69,22.76,2.0,1.0,1,...,2,0,0,0,1,0,0,1,1,1
3,1,0,77,110,68.0,1.59,26.90,3.0,1.0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,54,28,65.0,1.70,22.49,2.0,1.0,0,...,2,0,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,1,1,83,194,75.0,1.60,29.30,3.0,1.0,0,...,2,0,0,0,1,0,0,0,1,1
127,1,0,67,578,94.0,1.74,31.05,4.0,1.0,1,...,0,0,0,0,0,0,0,0,0,1
128,1,0,67,52,65.0,1.74,21.47,2.0,0.0,1,...,1,0,0,0,0,0,0,1,1,0
129,1,0,74,269,83.0,1.65,30.49,4.0,0.0,1,...,2,0,0,1,1,0,0,0,1,1


Preprocessament

In [5]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd

data1.columns = data1.columns.str.strip()

# Identificar columnes binàries (0 o 1)
binary_columns = [col for col in data1.columns if data1[col].dropna().unique().tolist() == [0, 1]]

# Identificar columnes categòriques,
# i excloure les binàries de la llista de categòriques
categorical_columns = [col for col in data1.columns if data1[col].dtype in ['object', 'category'] and col not in binary_columns]

# Identificar columnes ordinals (numèriques amb un nombre limitat de categories)
ordinal_columns = [col for col in data1.columns if (data1[col].dtype in ['int64', 'float64'] and len(data1[col].unique()) <= 10) and col not in binary_columns]

# Identificar columnes numèriques (que no siguin binàries, ordinals ni categòriques)
numerical_columns = [col for col in data1.columns if data1[col].dtype in ['int64', 'float64'] and len(data1[col].unique()) > 10 and col not in binary_columns + categorical_columns + ordinal_columns]

# Mostrar les columnes identificades per tipus
print("Columnes binàries:", binary_columns)
print("Columnes categòriques:", categorical_columns)
print("Columnes ordinals:", ordinal_columns)
print("Columnes numèriques:", numerical_columns)

# Crear un transformer per a les variables binàries, categòriques, ordinals i numèriques
preprocessor = ColumnTransformer(
    transformers=[
        ('bin', 'passthrough', binary_columns),       # No es fa cap canvi
        ('cat', OneHotEncoder(), categorical_columns),  # OneHotEncoding per a variables categòriques
        ('ord', 'passthrough', ordinal_columns),
        ('num', StandardScaler(), numerical_columns)    # StandardScaler per a variables numèriques
    ])

# Aplicar el transformer a les teves dades
X_processed = preprocessor.fit_transform(data1)



Columnes binàries: ['RECIST', 'Alcohol _abuse', 'Obesity', 'disorders', 'Drug-abuse/addict', 'Smoke', 'Diabetes', 'Cancer History', 'cancer_active', 'Statin_treatment', 'Antiretroviral_treatment', 'Cronic_Kidney_Insufficiency', 'PORTAL_HYPERTENSION_SIGNS', 'I', 'II', 'III', 'IVa', 'IVb', 'V', 'VI', 'VII']
Columnes categòriques: []
Columnes ordinals: ['GENDER', 'BMI_CATEGORY', 'HCV', 'Aflatoxin', 'no_active_ex', 'Hypertension', 'B-block_treatment', 'Kidney_substitute_treatment', 'Fibrosis_grade', 'ALBI_grade', 'Child_Pugh', 'IMAGE_MODALITY_FOR_LIRADS', 'IV_CONTRAST', 'TECHNICALLY_ACCEPTABLE_FOR_LIRADS', 'MORPHOLICAL_CHANGES_OF_HEPATIC_CIRRHOSIS', 'VENOUS_PATENCY', 'LI-RADS_VALORATION_MODALITY', 'WIDESPREAD_DISEASE', 'TIV', 'NUM_INDIV_OBS', 'LR1', 'TNM_8TH', 'Tumoral_Distribution', 'VIII']
Columnes numèriques: ['ID', 'CLAVE', 'AGE_AT_TACE', 'DAYS_CT_PRETACE', 'WEIGHT', 'HEIGHT', 'BMI', 'ALBI_Score', 'Meld_Original', 'Meld-Na', 'Globulos_blancos', 'Globulos_rojos', 'Hemoglobin', 'Hematocr

Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut
from imblearn.over_sampling import SMOTE

data1.columns = data1.columns.str.strip()

y = data1.loc[:, "RECIST"].values

X = data1.loc[:, (data1.columns != "RECIST") & (data1.columns != "ID") & (data1.columns != "CLAVE")].values
feature_names = data1.columns[(data1.columns != "RECIST") & (data1.columns != "ID") & (data1.columns != "CLAVE")]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
loo = LeaveOneOut()
model = LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear', C=1.0)

total_bootstrap_iterations = 1000
variable_counts = np.zeros(X.shape[1])

all_bootstrap_probabilities = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    bootstrapped_probabilities = []

    for _ in range(total_bootstrap_iterations):
        X_train_bootstrap, y_train_bootstrap = resample(X_train_smote, y_train_smote, replace=True)

        model.fit(X_train_bootstrap, y_train_bootstrap)

        prob = model.predict_proba(X_test)[:, 1][0]
        bootstrapped_probabilities.append(prob)

        # Actualitzar el contador per les variables que sí que hem seleccionat (coef != 0)
        variable_counts += model.coef_[0] != 0  # Sumam 1 per cada una d'elles

    all_bootstrap_probabilities.append(bootstrapped_probabilities)

df_probabilities = pd.DataFrame(all_bootstrap_probabilities)

variable_importance = pd.DataFrame({
    'Variable': feature_names,
    'Frequency (%)': (variable_counts / (total_bootstrap_iterations * loo.get_n_splits(X_scaled))) * 100  # Porcentaje de veces seleccionada
}).sort_values(by='Frequency (%)', ascending=False)


# Mostrar la importància de les variables
df_probabilities.head()


In [23]:
pd.set_option('display.max_rows', None)
print("\nImportancia de las variables (Count y Frequency):")
print(variable_importance)


Importancia de las variables (Count y Frequency):
                                    Variable  Frequency (%)
9                                    Obesity      96.392366
70                           MAX_TM_DIAMETER      96.079389
59         TECHNICALLY_ACCEPTABLE_FOR_LIRADS      94.799237
47                         Total_Bilirrubine      91.911450
20                          Statin_treatment      91.588550
77                                       IVb      90.329008
58                               IV_CONTRAST      89.648092
76                                       IVa      89.165649
8                             Alcohol _abuse      88.012977
81                                      VIII      86.667176
68                                   TNM_8TH      85.364122
32                                Hemoglobin      84.370229
16                              Hypertension      83.658779
78                                         V      82.931298
7                                        HCV     

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

In [11]:
df_probabilities = pd.DataFrame(all_bootstrap_probabilities)
df_probabilities.insert(0, 'ID', data1['ID'].values)
df_probabilities.to_csv("huse4bootstrap_probabilities_id.csv", index=False)

In [13]:
taula1 = pd.read_csv('huse4bootstrap_probabilities_id.csv')
taula2 = pd.read_csv('df_gower.csv')

taula2.drop(['CLAVE'],axis=1,inplace=True)

In [14]:

id_column = taula1.iloc[:, 0]

probabilities = taula1.iloc[:, 1:]

y_pred = (probabilities.mean(axis=1) >= 0.5).astype(int)

taula1_processed = pd.DataFrame({'ID': id_column, 'RECIST': y_pred})

taula2_processed = taula2[['ID', 'RECIST']]
taula2_processed.rename(columns={'RECIST': 'y_true'}, inplace=True)

taula05 = pd.merge(taula1_processed, taula2_processed, on='ID')

print(taula05.head())

    ID  RECIST  y_true
0  222       0       0
1  346       1       0
2  613       0       1
3  697       1       1
4  745       1       0


<ipython-input-14-d60078a473e9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taula2_processed.rename(columns={'RECIST': 'y_true'}, inplace=True)


In [15]:
y_pred = taula05['RECIST']
y_true = taula05['y_true']

cm = confusion_matrix(y_true, y_pred)
print("Matriu de confusió:")
print(cm)

precision = precision_score(y_true, y_pred)
print(f"Precisió: {precision:.2f}")

recall = recall_score(y_true, y_pred)
print(f"Recall: {recall:.2f}")

f1 = f1_score(y_true, y_pred)
print(f"F1-Score: {f1:.2f}")

accuracy = accuracy_score(y_true, y_pred)
print(f"Exactitut: {accuracy:.2f}")

Matriu de confusió:
[[ 6 15]
 [36 74]]
Precisió: 0.83
Recall: 0.67
F1-Score: 0.74
Exactitut: 0.61


In [17]:
df_data = pd.read_csv("df_gower.csv")
df_data.drop(['CLAVE'],axis=1,inplace=True)

df_probabilities = pd.read_csv("huse4bootstrap_probabilities_id.csv")

df_data_subset = df_data[['ID', 'RECIST']]

if len(df_data_subset) == len(df_probabilities):
    df_final = pd.concat([df_data_subset, df_probabilities], axis=1)

    df_final.to_csv("final_table.csv", index=False)

    print("Tabla final creada y guardada en 'final_table.csv'.")
else:
    print("Error: Los DataFrames tienen diferente número de filas. Verifica tus datos.")

Tabla final creada y guardada en 'final_table.csv'.


In [18]:
df = pd.read_csv('final_table.csv')

id_column = 'ID'
diagnosis_column = 'RECIST'
prob_columns = df.columns[3:]

def calcular_intervalo(probabilidades, li, ls):
    lower_bound = np.percentile(probabilidades, li)
    upper_bound = np.percentile(probabilidades, ls)
    return lower_bound, upper_bound

In [19]:
intervalos_confianza = []

for index, row in df.iterrows():
    probabilidades = row[prob_columns].values
    lower, upper = calcular_intervalo(probabilidades, 2.5, 97.5)
    intervalos_confianza.append([row[id_column], row[diagnosis_column], lower, upper])

df_intervalos = pd.DataFrame(intervalos_confianza, columns=[id_column, diagnosis_column, 'lower_bound', 'upper_bound'])

In [20]:
def calcular_TP(a, b):
    TP = min(1 - a, b)
    return TP
def calcular_AP(a,b):
    AP = min(a, 1-b)
    return AP
def calcular_EP(a,b):
    EP = (b-a)
    return EP

In [21]:
df_intervalos['TP'] = df_intervalos.apply(lambda row: calcular_TP(row['lower_bound'], row['upper_bound']), axis=1)

df_intervalos['AP'] = df_intervalos.apply(lambda row: calcular_AP(row['lower_bound'], row['upper_bound']), axis=1)

df_intervalos['EP'] = df_intervalos.apply(lambda row: calcular_EP(row['lower_bound'], row['upper_bound']), axis=1)

In [22]:
medias = df_intervalos[['TP', 'AP', 'EP']].mean()
print("Medias:")
print(medias)

Medias:
TP    0.731555
AP    0.001201
EP    0.730354
dtype: float64
